## In this notebook I replicate Comma.ai Simulator

In [1]:
import sys
import numpy as np
import pygame

pygame 2.0.1 (SDL 2.0.14, Python 3.9.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import cv2
import glob
import pandas as pd

In [3]:
import random

In [4]:
path = '/home/naji/Desktop/fastai_final_project/datasets/Udacity_Real_Data/4-raw-data/Ch2_001/'

### Get perspective transform for images

In [5]:
from skimage import transform as tf

In [6]:
rsrc = [  [43.45456230828867, 118.00743250075844],
          [104.5055617352614, 69.46865203761757],
          [114.86050156739812, 60.83953551083698],
          [129.74572757609468, 50.48459567870026],
          [132.98164627363735, 46.38576532847949],
          [301.0336906326895, 98.16046448916306],
          [238.25686790036065, 62.56535881619311],
          [227.2547443287154, 56.30924933427718],
          [209.13359962247614, 46.817221154818526],
          [203.9561297064078, 43.5813024572758]]

In [7]:
rdst = [  [10.822125594094452, 1.42189132706374],
          [21.177065426231174, 1.5297552836484982],
          [25.275895776451954, 1.42189132706374],
          [36.062291434927694, 1.6376192402332563],
          [40.376849698318004, 1.42189132706374],
          [11.900765159942026, -2.1376192402332563],
          [22.25570499207874, -2.1376192402332563],
          [26.785991168638553, -2.029755283648498],
          [37.033067044190524, -2.029755283648498],
          [41.67121717733509, -2.029755283648498]]

In [8]:
tform3_img = tf.ProjectiveTransform()

In [9]:
tform3_img.estimate(np.array(rdst), np.array(rsrc))

True

In [10]:
def perspective_tform(x, y):
    p1, p2 = tform3_img((x,y))[0]
    return p2, p1

### Functions to draw lines 

In [11]:
def draw_pt(img, x, y, color, sz=1):
    row, col = perspective_tform(x, y)
    row = int(row)
    col = int(col)
    if row >= 0 and row < img.shape[0] and col >= 0 and col < img.shape[1]:
        img[row-sz:row+sz, col-sz:col+sz] = color

In [12]:
def draw_path(img, path_x, path_y, color):
    for x, y in zip(path_x, path_y):
        draw_pt(img, x, y, color)

### Functions to draw predicted path 

In [13]:
def calc_curvature(v_ego, angle_steers, angle_offset=0):
    deg_to_rad = np.pi/180.
    slip_fator = 0.0014 # slip factor obtained from real data
    steer_ratio = 15.3  # from http://www.edmunds.com/acura/ilx/2016/road-test-specs/
    wheel_base = 2.67   # from http://www.edmunds.com/acura/ilx/2016/sedan/features-specs/

    angle_steers_rad = (angle_steers - angle_offset) * deg_to_rad
    curvature = angle_steers_rad/(steer_ratio * wheel_base * (1. + slip_fator * v_ego**2))
    return curvature

In [14]:
#*** this function returns the lateral offset given the steering angle, speed and the lookahead distance
def calc_lookahead_offset(v_ego, angle_steers, d_lookahead, angle_offset=0):
    curvature = calc_curvature(v_ego, angle_steers, angle_offset)
    # clip is to avoid arcsin NaNs due to too sharp turns
    y_actual = d_lookahead * np.tan(np.arcsin(np.clip(d_lookahead * curvature, -0.999, 0.999))/2.)
    return y_actual, curvature

In [15]:
def draw_path_on(img, speed_ms, angle_steers, color=(0,0,255)):
    path_x = np.arange(0., 50.1, 0.5)
    path_y, _ = calc_lookahead_offset(speed_ms, angle_steers, path_x)
    draw_path(img, path_x, path_y, color)

### Main loop

In [16]:
df = pd.read_csv(path + 'CH2_final_evaluation.csv')

In [17]:
df

,frame_id,steering_angle,public
0,1479425441182877835,-0.016928,0
1,1479425441232704425,-0.015708,0
2,1479425441282730750,-0.015708,0
3,1479425441332806714,-0.015708,1
4,1479425441382790272,-0.015708,0
...,...,...,...
5609,1479425721681585081,-0.089012,1
5610,1479425721731673391,-0.087266,0
5611,1479425721781662176,-0.088026,1
5612,1479425721831580656,-0.087266,1


In [18]:
files = list(df['frame_id'])
len(files)

5614

In [19]:
real_angles = list(df['steering_angle'])
len(real_angles)

5614

In [20]:
df2 = pd.read_csv(path + 'rambo.csv')

In [21]:
df2

,frame_id,steering_angle
0,1479425441182877835,-0.004179
1,1479425441232704425,-0.004179
2,1479425441282730750,-0.066690
3,1479425441332806714,-0.024831
4,1479425441382790272,-0.002446
...,...,...
5609,1479425721681585081,-0.068208
5610,1479425721731673391,-0.079429
5611,1479425721781662176,-0.128137
5612,1479425721831580656,-0.101860


In [22]:
predict_angles = list(df2['steering_angle'])
len(predict_angles)

5614

In [39]:
data = zip(files, real_angles, predict_angles)

In [40]:
pygame.init()
size = (320*2, 160*2)
pygame.display.set_caption("Naji data viewer")
screen = pygame.display.set_mode(size, pygame.DOUBLEBUF)

surface = pygame.surface.Surface((320,160),0,24).convert()

In [41]:
running = True

In [42]:
for file, real_angle, predict_angle in data:
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    
    if running:      
        np_img = cv2.imread(path + 'HMB_3_release/center/' + str(file) + '.jpg')
        resized_img2 = cv2.resize(np_img, (int(np_img.shape[1]*0.5), int(np_img.shape[0]*0.335)) )

        predicted_steers = random.randint(20,30)    
        angle_steers = real_angle    
        speed_ms = random.randint(25,30)

        draw_path_on(resized_img2, speed_ms, -angle_steers/10.0, (0, 255, 0))
        
        draw_path_on(resized_img2, speed_ms, -predicted_steers/10.0, (255, 0, 0))

        pygame.surfarray.blit_array(surface, resized_img2.swapaxes(0,1))
        camera_surface_2x = pygame.transform.scale2x(surface)
        screen.blit(camera_surface_2x, (0,0))    
        pygame.display.flip()    
        # pygame.time.Clock().tick(60)
    
    else:     
        pygame.quit()
        break
